In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('../data/ds_test_training_dataset_2024.csv')
test = pd.read_csv('../data/ds_test_houston_2024.csv')

print(f"Train: {train.shape}, Test: {test.shape}")

Train: (7462, 49), Test: (2539, 48)


In [2]:
train.columns

Index(['id', 'state', 'county_name', 'media_market', 'gender',
       'political_party', 'education_area', 'age', 'gender_female',
       'gender_male', 'income', 'maritalstatus_single',
       'maritalstatus_married', 'length_of_residence', 'has_children',
       'number_of_children', 'is_homeowner', 'is_renter',
       'Interest in Barbeque', 'religion_catholic', 'religion_christian',
       'donor_political_org', 'donor_liberal_org', 'donor_conservative_org',
       'donor_religious_org', 'donor_health_org', 'occupation_blue_collar',
       'occupation_farmer', 'occupation_professional_technical',
       'occupation_retired', 'purchases_apparel', 'purchases_book',
       'purchases_electronic', 'purchases_boat', 'purchases_luxuryitems',
       'has_a_cat', 'interests_environment', 'interests_outdoorgarden',
       'interests_outdoorsport', 'interests_guns', 'interests_golf',
       'interests_investing', 'interests_veteranaffairs', 'ethnicity_afam',
       'ethnicity_latino', 'ethni

In [3]:
train['Plan Enrolled'].value_counts()

Plan Enrolled
Not Subscribed    3764
Plan Blue         2536
Plan Red          1162
Name: count, dtype: int64

In [4]:
# create target and combine datasets
train['is_enrolled'] = (train['Plan Enrolled'] != 'Not Subscribed').astype(int)
train['is_train'] = 1
test['is_train'] = 0

df = pd.concat([train, test], ignore_index=True)
print(f"Combined: {df.shape}")

Combined: (10001, 51)


In [5]:
df[df.state.isna()]

,id,state,county_name,media_market,gender,political_party,education_area,age,gender_female,gender_male,...,interests_investing,interests_veteranaffairs,ethnicity_afam,ethnicity_latino,ethnicity_asian,ethnicity_white,ethnicity_other,Plan Enrolled,is_enrolled,is_train
7461,(10000 rows),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Subscribed,0.0,1


Weird... ill just drop this one row

In [6]:
df.drop(index=df[df.state.isna()].index, inplace=True)
df.isna().sum()

id                                      0
state                                   0
county_name                             0
media_market                            0
gender                                  0
political_party                         0
education_area                          0
age                                     0
gender_female                           0
gender_male                             0
income                                  0
maritalstatus_single                    0
maritalstatus_married                   0
length_of_residence                     0
has_children                            0
number_of_children                      0
is_homeowner                            0
is_renter                               0
Interest in Barbeque                    0
religion_catholic                       0
religion_christian                      0
donor_political_org                     0
donor_liberal_org                       0
donor_conservative_org            

In [7]:
# encode bbq interest as ordinal
bbq_map = {
    'No interest': 0,
    'Low Interest': 1, 
    'Moderate Interest': 2,
    'High Interest': 3,
    'Very High Interest': 4
}
df['bbq_interest'] = df['Interest in Barbeque'].map(bbq_map)

In [8]:
# encode education area
edu_map = {
    '01-High Education Area': 3,
    '02-Medium Education Area': 2,
    '03-Low Education Area': 1
}
df['education_level'] = df['education_area'].map(edu_map)

# political party dummies
df['is_republican'] = (df['political_party'] == 'R').astype(int)
df['is_democrat'] = (df['political_party'] == 'D').astype(int)
df['is_independent'] = (df['political_party'] == 'I').astype(int)

In [9]:
# aggregate interest flags
interest_cols = ['interests_environment', 'interests_outdoorgarden', 'interests_outdoorsport',
                 'interests_guns', 'interests_golf', 'interests_investing', 'interests_veteranaffairs']
df['n_interests'] = df[interest_cols].sum(axis=1)

# aggregate purchase flags  
purchase_cols = ['purchases_apparel', 'purchases_book', 'purchases_electronic', 
                 'purchases_boat', 'purchases_luxuryitems']
df['n_purchases'] = df[purchase_cols].sum(axis=1)

# aggregate donor flags
donor_cols = ['donor_political_org', 'donor_liberal_org', 'donor_conservative_org',
              'donor_religious_org', 'donor_health_org']
df['n_donor_types'] = df[donor_cols].sum(axis=1)
df['is_any_donor'] = (df['n_donor_types'] > 0).astype(int)

In [10]:
sorted(df.income.values)[-1]

150

In [12]:
df.age.isna().sum()

0

In [13]:
df['children_x_income'] = df['has_children'] * df['income']                                                                                                                                                                                                             
df['age_x_homeowner'] = df['age'] * df['is_homeowner']                                                                                                                                                                                                                  
df['married_x_children'] = df['maritalstatus_married'] * df['has_children']                                                                                                                                                                                             
                                                                                                                                                                                                                                                                        
# age bins                                                                                                                                                                                                                                                              
df['age_bin'] = pd.cut(df['age'], bins=[0, 30, 45, 60, 200], labels=[0, 1, 2, 3]).astype(int)                                                                                                                                                                           
                                                                                                                                                                                                                                                                        
# income features using external benchmarks                                                                                                                                                                                                                             
# sources:                                                                                                                                                                                                                                                              
#   - Houston median household income ~$61K (census.gov ACS 2023)                                                                                                                                                                                                       
#   - Texas statewide median ~$67K                                                                                                                                                                                                                                      
HOUSTON_MEDIAN_INCOME = 61                                                                                                                                                                                                                                              
TEXAS_MEDIAN_INCOME = 67                                                                                                                                                                                                                                                

# engineer some features that are relative to houston and texas.                                                                                                                                                                                                                   
df['income_vs_houston'] = df['income'] / HOUSTON_MEDIAN_INCOME  # ratio to houston median                                                                                                                                                                               
df['income_vs_texas'] = df['income'] / TEXAS_MEDIAN_INCOME                                                                                                                                                                                                              
df['above_houston_median'] = (df['income'] > HOUSTON_MEDIAN_INCOME).astype(int)                                                                                                                                                                                         
df['above_texas_median'] = (df['income'] > TEXAS_MEDIAN_INCOME).astype(int)                                                                                                                                                                                             
                                                                                                                                                                                                                                                                        
# income bins based on houston context: low / median / comfortable / high                                                                                                                                                                                               
df['income_bin'] = pd.cut(df['income'], bins=[-np.inf, 45, 75, 120, np.inf], labels=[0, 1, 2, 3]).astype(int)

### Bit more feature engineering

In [14]:
# life stage and financial behavior flags

# young family - prime dependent care FSA candidates (as per google's suggestions)
df['is_young_family'] = (
    (df['maritalstatus_married'] == 1) & 
    (df['has_children'] == 1) & 
    (df['age'] < 45)
).astype(int)

# established - stable, settled, likely to plan ahead financially and also have more financial resources to pay
df['is_established'] = (
    (df['is_homeowner'] == 1) & 
    (df['length_of_residence'] > 10) & 
    (df['maritalstatus_married'] == 1)
).astype(int)

# pre-retirement - HSA as retirement savings vehicle, also as they grow older they are more in need of a HSA so they dont need to pay from their pcokets / 401k

df['is_pre_retirement'] = (
    (df['age'] >= 50) & 
    (df['age'] <= 65) & 
    (df['income'] > HOUSTON_MEDIAN_INCOME)
).astype(int)

# financially savvy - understands tax-advantaged accounts, also potentially more money to save
df['financially_savvy'] = (
    (df['interests_investing'] == 1) & 
    (df['income'] > HOUSTON_MEDIAN_INCOME) & 
    (df['education_level'] >= 2)
).astype(int)

In [21]:
# check distributions of new flags
print(df['is_young_family'].value_counts() / len(df))
print(df['is_established'].value_counts() / len(df))
print(df['is_pre_retirement'].value_counts() / len(df))
print(df['financially_savvy'].value_counts() / len(df))

is_young_family
0    0.6837
1    0.3163
Name: count, dtype: float64
is_established
0    0.6355
1    0.3645
Name: count, dtype: float64
is_pre_retirement
0    0.685
1    0.315
Name: count, dtype: float64
financially_savvy
0    0.7716
1    0.2284
Name: count, dtype: float64


Maybe these features can have some predictive power, we shall see in the next notebooks :)

In [23]:
# keep id separate for final output, then drop non-predictive columns
ids = df['id'].copy()

drop_cols = [
    'id', 'state', 'county_name', 'media_market',  # identifiers
    'gender', 'political_party', 'education_area', 'Interest in Barbeque',  # already encoded
    'Plan Enrolled'  # target-related
]
df = df.drop(columns=drop_cols)

In [24]:
# split back to train/test
train_df = df[df['is_train'] == 1].drop(columns=['is_train']).reset_index(drop=True)
test_df = df[df['is_train'] == 0].drop(columns=['is_train', 'is_enrolled']).reset_index(drop=True)
test_ids = ids[df['is_train'] == 0].reset_index(drop=True)

print(f"Train: {train_df.shape}, Test: {test_df.shape}")
print(f"\nFeatures: {list(train_df.columns)}")

Train: (7461, 63), Test: (2539, 62)

Features: ['age', 'gender_female', 'gender_male', 'income', 'maritalstatus_single', 'maritalstatus_married', 'length_of_residence', 'has_children', 'number_of_children', 'is_homeowner', 'is_renter', 'religion_catholic', 'religion_christian', 'donor_political_org', 'donor_liberal_org', 'donor_conservative_org', 'donor_religious_org', 'donor_health_org', 'occupation_blue_collar', 'occupation_farmer', 'occupation_professional_technical', 'occupation_retired', 'purchases_apparel', 'purchases_book', 'purchases_electronic', 'purchases_boat', 'purchases_luxuryitems', 'has_a_cat', 'interests_environment', 'interests_outdoorgarden', 'interests_outdoorsport', 'interests_guns', 'interests_golf', 'interests_investing', 'interests_veteranaffairs', 'ethnicity_afam', 'ethnicity_latino', 'ethnicity_asian', 'ethnicity_white', 'ethnicity_other', 'is_enrolled', 'bbq_interest', 'education_level', 'is_republican', 'is_democrat', 'is_independent', 'n_interests', 'n_purch

In [25]:
# sanity check
print("Target distribution:")
print(train_df['is_enrolled'].value_counts(normalize=True))

print(f"\nMissing values in train: {train_df.isnull().sum().sum()}")

Target distribution:
is_enrolled
0.0    0.504356
1.0    0.495644
Name: proportion, dtype: float64

Missing values in train: 0


In [26]:
# prep X and y for modeling
feature_cols = [c for c in train_df.columns if c != 'is_enrolled']
X_train = train_df[feature_cols]
y_train = train_df['is_enrolled']
X_test = test_df[feature_cols]

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}")

X_train: (7461, 62), y_train: (7461,), X_test: (2539, 62)


In [27]:
# save for modeling notebook
X_train.to_csv('../data/modeling/X_train.csv', index=False)
y_train.to_csv('../data/modeling/y_train.csv', index=False)
X_test.to_csv('../data/modeling/X_test.csv', index=False)
test_ids.to_csv('../data/modeling/test_ids.csv', index=False)

print("Saved to ../data/modeling/")

Saved to ../data/modeling/
